In [1]:

from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.callback import CallbackCollection, Callback
from pymoo.core.problem import ElementwiseProblem, Problem
from pymoo.problems.functional import FunctionalProblem, func_return_none
from pymoo.optimize import minimize
from pymoo.problems.dyn import TimeSimulation
from pymoo.problems.dynamic.df import DF1
from pymoo.problems.multi import CTP3
from pymoo.problems.multi.srn import SRN
import matplotlib.pyplot as plt

import zuko
from zuko.flows import GMM
import numpy as np
import torch

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
robot = Robot(True)

ERobot: kuka_lbr_iiwa_14_r820 (by Kuka), 7 joints (RRRRRRR), 2 branches, geometry, collision
┌─────┬───────────┬───────┬───────────┬───────────────────────────────────┐
│link │   link    │ joint │  parent   │        ETS: parent to link        │
├─────┼───────────┼───────┼───────────┼───────────────────────────────────┤
│   0 │ base_link │       │ BASE      │ SE3()                             │
│   1 │ link_1    │     0 │ base_link │ SE3() ⊕ Rz(q0)                    │
│   2 │ link_2    │     1 │ link_1    │ SE3(-0.0004362, 0, 0.36) ⊕ Ry(q1) │
│   3 │ link_3    │     2 │ link_2    │ SE3() ⊕ Rz(q2)                    │
│   4 │ link_4    │     3 │ link_3    │ SE3(0.0004362, 0, 0.42) ⊕ Ry(-q3) │
│   5 │ link_5    │     4 │ link_4    │ SE3() ⊕ Rz(q4)                    │
│   6 │ link_6    │     5 │ link_5    │ SE3(0, 0, 0.4) ⊕ Ry(q5)           │
│   7 │ link_7    │     6 │ link_6    │ SE3() ⊕ Rz(q6)                    │
│   8 │ @tool0    │       │ link_7    │ SE3(0, 0, 0.126)               

In [2]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda, num_samples=config.num_train_size)
# build dimension reduction model
hnne = get_hnne_model(X, y)
# get loader
loader = get_loader(X, y, hnne=hnne)
# get val loader
val_loader = get_val_loader(robot=panda, hnne=hnne)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [3]:
mu = torch.randn(size=(7,))
print(mu)
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0)
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=iflow, 
                      num_data=1, 
                      num_samples=1000)

tensor([ 3.1589,  0.2793, -0.3395,  0.8086,  0.4431, -0.9373,  1.9431])


In [4]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(robot=panda, 
                      loader=val_loader, 
                      model=nflow, 
                      num_data=100, 
                      num_samples=1000)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')

In [29]:
class ProblemWrapper(Problem):
    
    def __init__(self, 
                 end_effector_path,
                 model,
                 step_mod = 10,
                 l2_love_ratio = 0.3,
                 num_samples = 3,
                 **kwargs):
        self.end_effector_path = end_effector_path
        self.model = model
        self.step = 0
        self.step_mod = step_mod
        self.l2_love_ratio = l2_love_ratio
        self.constr_ieq = []
        self.num_samples = num_samples
        self.robot = Robot(verbose=False)
        self.solutions = np.zeros((len(end_effector_path), self.robot.dof))
        xl, xu = self.__x_bounds()
        super().__init__(n_var=len(xl),
                         n_obj=2,
                         n_ieq_constr=len(self.constr_ieq),
                         n_eq_constr=0,
                         xl=xl,
                         xu=xu,
                         **kwargs)
        
    def iksols(self, x):
        # need to check the output shapes for generating solutions
        pos = np.atleast_2d(self.pos()).repeat(repeats=len(x), axis=0)
        noise = np.zeros((len(pos),))
        x = torch.tensor(np.column_stack((pos, x, noise)), dtype=torch.float32).to(config.device)
        out = self.model(x).sample((self.num_samples,))
        out = out.detach().cpu().numpy()
        out = out.reshape((-1, self.robot.dof))
        return out
        
    def l2_err(self, iksols):
        errs = self.robot.l2_err_func_array(qs=iksols, ee_pos=self.pos())
        return errs
    
    def ang_jump(self, iksols):
        if self.index() == 0:
            return np.zeros((len(iksols), ))
        errs = (np.abs(iksols - self.solutions[self.index()-1])).sum(axis=1)
        return errs

    def __x_bounds(self):
        X_trans = load_numpy(file_path=config.x_trans_train_path)
        df = pd.DataFrame(data=X_trans)
        return df.min(axis=0).values, df.max(axis=0).values
    
    def pos(self):
        return self.end_effector_path[self.index()]
    
    def index(self):
        return self.step // self.step_mod
    
    def tic(self):
        self.step += 1
    
    def prune_samples(self, iksols, errs):
        ik_boxes = iksols.reshape((self.num_samples, -1, iksols.shape[-1]))
        err_boxes = errs.reshape((self.num_samples, -1, errs.shape[-1]))
        
        pop_size = ik_boxes.shape[1]
        survivors = np.zeros((pop_size, iksols.shape[-1]))
        surv_errs = np.zeros((pop_size, errs.shape[-1]))
        
        for pi in range(pop_size):
            survivors[pi], surv_errs[pi] = self.__get_best_iksols(iksols=ik_boxes[:, pi], errs=err_boxes[:, pi])
        return survivors, surv_errs
            
    def __get_best_iksols(self, iksols, errs):
        idx = np.argmin(self.love_index(errs))
        return iksols[idx], errs[idx]
    
    def love_index(self, errs):
        return errs[:, 0] * self.l2_love_ratio + errs[:, 1] * (1 - self.l2_love_ratio)
        
    def __set_best_iksols(self, iksols, errs):
        self.solutions[self.index()], _ = self.__get_best_iksols(iksols, errs)

    def _evaluate(self, x, out, *args, **kwargs):
        iksols = self.iksols(x)
        l2err = self.l2_err(iksols)
        ang_jump = self.ang_jump(iksols)
        errs = np.column_stack((l2err.reshape(-1, 1), ang_jump.reshape(-1, 1)))
        iksols, errs = self.prune_samples(iksols, errs)
        self.__set_best_iksols(iksols, errs)
        out["F"] = errs

In [30]:
traj_dir = sample_ee_traj(robot=panda, load_time='')
ee_traj = load_numpy(file_path=traj_dir + 'ee_traj.npy')
q_traj = load_numpy(file_path=traj_dir + 'q_traj.npy')

100%|██████████| 20/20 [00:00<00:00, 15293.72it/s]

mkdir /home/luca/ikpflow/data/panda/trajectory/06032319
/home/luca/ikpflow/data/panda/trajectory/06032319/ load successfully.


In [33]:
from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.core.callback import CallbackCollection, Callback
from pymoo.optimize import minimize
from pymoo.problems.dyn import TimeSimulation
from pymoo.problems.dynamic.df import DF1
import matplotlib.pyplot as plt


problem = ProblemWrapper(end_effector_path=ee_traj, 
                         model=nflow, 
                         step_mod=10,
                         l2_love_ratio=0.1)

algorithm = DNSGA2(pop_size=20)

simulation = TimeSimulation()

res = minimize(problem,
               algorithm,
               termination=('n_gen', len(ee_traj)*problem.step_mod),
               callback=CallbackCollection(simulation),
               seed=1,
               verbose=True)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       20 |      1 |             - |             -
     2 |       82 |      1 |  0.0004196973 |             f
     3 |      144 |      1 |  0.0007720587 |             f
     4 |      206 |      1 |  0.0007959580 |             f
     5 |      268 |      1 |  0.0006378504 |             f
     6 |      330 |      1 |  0.0006164345 |             f
     7 |      392 |      1 |  0.0006491318 |             f
     8 |      454 |      1 |  0.0007686029 |             f
     9 |      516 |      1 |  0.0007898105 |             f
    10 |      578 |      1 |  0.0007417715 |             f
    11 |      640 |      3 |  0.9296457154 |         ideal
    12 |      702 |      6 |  0.1797334654 |         ideal
    13 |      764 |      4 |  0.0321935268 |         ideal
    14 |      826 |      2 |  0.7673639788 |         ideal
    15 |      888 |      4 |  0.0378279629 |         ideal
    16 |      950 |      3 |  0.0680980926 |         ide

In [35]:
def cal_errs(robot = problem.robot, path = ee_traj, iksols = problem.solutions):
    errs = np.zeros((len(path),))
    ang_errs = np.zeros_like(errs)
    step = 0
    for q, ee_pos in zip(iksols, path):
        if step != 0:
            ang_errs[step] = np.sum(np.abs(iksols[step-1] - q))
        errs[step] = robot.l2_err_func(q=q, ee_pos=ee_pos)
        step += 1
    ang_errs[0] = np.average(ang_errs[1:])
    ang_errs = np.rad2deg(ang_errs)
    df = pd.DataFrame(np.column_stack((errs, ang_errs)), columns=['l2_err', 'ang_errs(sum)'])
    return df
df = cal_errs()
print(df.describe())

          l2_err  ang_errs(sum)
count  20.000000      20.000000
mean    0.005115      79.052103
std     0.006266      53.771811
min     0.000530      18.118501
25%     0.002128      49.221672
50%     0.003435      70.894992
75%     0.005656      79.912182
max     0.029479     231.791509


In [ ]:
z_dim = 3
c_dim = 3
model = GMM(features=z_dim, context=c_dim, hidden_features=[10], components=5)
print(model)
bs = 100
input_z = torch.zeros((bs, z_dim,))
input_c = torch.zeros((bs, c_dim,))
ans = model(input_c).sample((2,)).numpy()
print(ans.shape)
print(ans)
reshape = ans.reshape((-1, ans.shape[-1]))
print(reshape.shape)
print(reshape)

In [ ]:
pos = np.array([[1, 2, 3], [4, 5, 6]])

In [ ]:
np.atleast_2d(pos).repeat(repeats=len(reshape), axis=0)

In [ ]:
problem = FunctionalProblem(z_dim,
                            objs,
                            xl=np.array([-10, -5, -10]),
                            xu=np.array([10, 5, 10])
                            )

algorithm = NSGA2()

res = minimize(problem,
               algorithm,
               termination=('n_gen', 200),
               seed=1,
               verbose=True)

In [ ]:
flow = MAF(3, 4, transforms=3, passes=2) # coupling layers
flow

In [ ]:
flow = MAF(features=config.num_features, 
            context=config.num_conditions, 
            transforms=config.num_transforms, 
            passes=2,
            randperm=True, 
            activation=config.activation, 
            hidden_features=config.subnet_shape).to(config.device)
flow

In [ ]:
import roboticstoolbox as rbt

In [ ]:
import spatialmath as spm

In [ ]:
p = spm.SE3(*[1, 4, 2])
p

In [ ]:
p.angvec()

In [ ]:
os.path.dirname(p)

In [ ]:
def load_numpy(file_path):
    if os.path.exists(path=file_path):
        return np.load(file_path)
    else:
        print('file not exist and return empty np array.')
        return np.array([])

In [ ]:
from utils.utils import create_robot_dirs, remove_empty_robot_dirs

In [ ]:
create_robot_dirs()

In [ ]:
remove_empty_robot_dirs()

In [ ]:
import numpy as np

In [ ]:
k = np.zeros(shape=(4, 4))

In [ ]:
k.shape == (4,4)

In [ ]:
from utils.robot import Robot

In [ ]:
rbt = Robot(verbose=False)

In [ ]:
rbt.robot.name

In [ ]:
print(rbt.robot)

In [ ]:
q = rbt.robot.qrandom
rbt.plot(q=q, p=q)

In [ ]:
import roboticstoolbox as rtb

In [ ]:
rtb.models.DH.__all__

In [ ]:
robot = rtb.models.DH.KR5()

In [ ]:
robot.qlim

In [ ]:
robot.qlim.shape

In [ ]:
its = .7
robot.plot(q=robot.qrandom, jointaxes=False, limits=[-its, its] * 3)

In [ ]:
import time

In [ ]:
b = time.time()

In [ ]:
e = time.time()

In [ ]:
round(e - b, 2)

In [ ]:
import numpy as np
k = np.zeros((7, 7))

In [ ]:
u = k.reshape(-1)

In [ ]:
u.shape

In [1]:
import swift
import roboticstoolbox as rtb
import spatialmath as sm
import numpy as np

env = swift.Swift()
env.launch(realtime=True)

panda = rtb.models.Panda()
panda.q = panda.qr

Tep = panda.fkine(panda.q) * sm.SE3.Trans(0.2, 0.2, 0.45)

arrived = False
env.add(panda)

dt = 0.05

while not arrived:

    v, arrived = rtb.p_servo(panda.fkine(panda.q), Tep, 1)
    panda.qd = np.linalg.pinv(panda.jacobe(panda.q)) @ v
    env.step(dt)

# Uncomment to stop the browser tab from closing
# env.hold()

In [11]:
def run(robot, steps=20):
    env = swift.Swift()
    env.launch(realtime=True)
    env.add(robot)
    
    robot.q = robot.qr
    
    q = robot.q
    p = robot.random_q()
    
    qt = rtb.tools.trajectory.jtraj(q, p, t=steps)
    
    # robot.plot(qt.q)
    dt = 0.05

    for q in qt.q:
        robot.q = q
        env.step(dt)
    

In [10]:
panda = rtb.models.Panda()

run(panda)

In [ ]:
robot.plot()